In [11]:
# to clean and pre-process the data
# import data using pandas
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [12]:
# label the cluster map
# labels:
# region_hash, region_id

columns = ['region_hash', 'region_id']
# read the cluster map
cluster_map = pd.read_csv('../dataset/test_set/cluster_map/cluster_map', sep='\t', on_bad_lines='skip', header=None, names=columns)
print('cluster_map finished')

print('cluster_map.head(): \n', cluster_map.head())


cluster_map.to_csv('../dataset/labeledTestSet/cluster_map.csv', index=False)



cluster_map finished
cluster_map.head(): 
                         region_hash  region_id
0  90c5a34f06ac86aee0fd70e2adce7d8a          1
1  f2c8c4bb99e6377d21de71275afd6cd2          2
2  58c7a4888306d8ff3a641d1c0feccbe3          3
3  b26a240205c852804ff8758628c0a86a          4
4  4b9e4cf2fbdc8281b8a1f9f12b80ce4d          5


In [13]:
# label the orders data
# labels:
# order_id, driver_id, passenger_id, start_district_hash, dest_district_hash, price, time

columns = ['order_id', 'driver_id', 'start_region_hash', 'dest_region_hash', 'time']

# read the orders data
orders_data = []
for f in glob.glob('../dataset/test_set/order_data/test_order_data_*'):
    # file name
    print('filename: ', f)
    df = pd.read_csv(f, sep=',', on_bad_lines='skip', header=None, names=columns)
    orders_data.append(df)

print('orders_data finished')
orders_data = pd.concat(orders_data,  ignore_index=True)

# print('orders_data.head(): ', orders_data.head())
orders_data.to_csv('../dataset/labeledTestSet/orders_data.csv', index=False)


filename:  ../dataset/test_set/order_data\test_order_data_2016-01-23
filename:  ../dataset/test_set/order_data\test_order_data_2016-01-25
filename:  ../dataset/test_set/order_data\test_order_data_2016-01-27
filename:  ../dataset/test_set/order_data\test_order_data_2016-01-29
filename:  ../dataset/test_set/order_data\test_order_data_2016-01-31
orders_data finished


In [14]:
# label the weather data
# labels:
# time, weather, temperature, pm25
columns = ['time', 'weather', 'temperature', 'pm25']

# print('weather_data.head(): \n', weather_data.head())


# # read the weather data
weather_data = []
for f in glob.glob('../dataset/test_set/weather_data/weather_data_*'):
    # file name
    print('filename: ', f)
    df = pd.read_csv(f, sep='\t', on_bad_lines='skip', header=None, names=columns)
    weather_data.append(df)

print('weather_data finished')
weather_data = pd.concat(weather_data, ignore_index=True)


weather_data.to_csv('../dataset/labeledTestSet/weather_data.csv', index=False)


filename:  ../dataset/test_set/weather_data\weather_data_2016-01-23_test
filename:  ../dataset/test_set/weather_data\weather_data_2016-01-25_test
filename:  ../dataset/test_set/weather_data\weather_data_2016-01-27_test
filename:  ../dataset/test_set/weather_data\weather_data_2016-01-29_test
filename:  ../dataset/test_set/weather_data\weather_data_2016-01-31_test
weather_data finished


In [15]:
# label the poi data
# labels:
# region_hash, poi_id 
# 1st column: district_hash
# whole next column is: poi_id
columns = ['region_hash', 'poi_id']



# read the poi data
poi_data = pd.read_csv('../dataset/test_set/poi_data/poi_data', sep='\t', header=None, on_bad_lines='skip')

# extract the district_hash column and the POI ID columns
district_hash = poi_data.iloc[:, 0]
poi_ids = poi_data.iloc[:, 1:]

# combine all the POI IDs for each row into a list
poi_ids_list = poi_ids.apply(lambda x: x.tolist(), axis=1)

# combine the district_hash and poi_ids_list into a new DataFrame
labeled_poi_data = pd.concat([district_hash, poi_ids_list], axis=1)
labeled_poi_data.columns = ['region_hash', 'poi_ids']

# print the result
# print(labeled_poi_data.head())

# updated list
updated_list = []

# convert the column of lists to a list of lists
list_of_lists_poi_id = labeled_poi_data['poi_ids'].tolist()

# poi format poi_id = class:numofFacilities
# seperate numofFacilities from list_of_lists_poi_id and sum them up

# for each list in list_of_lists_poi_id 
# change the list of poi_id to sum of numofFacilities 
for poi_list in list_of_lists_poi_id:
    weighted_sum = 0
    for poi in poi_list:
        if(pd.isna(poi)==False):
            poi_id, num_of_facilities = poi.split(':')
            poi_class = poi_id.split('#')
            # combine the class1 and class2 numbers
            if(len(poi_class) == 1):
                poi_class[0] = '0' + poi_class[0]
            else:
                poi_number = poi_class[0] + '' + poi_class[1]
            weighted_sum += int(num_of_facilities) * int(poi_number)

    updated_list.append(weighted_sum)

# print(list_of_lists_poi_id)

# change labeled_poi_data['poi_ids'] to list_of_lists_poi_id
labeled_poi_data['poi_ids'] = updated_list

print('labeled_poi_data.head(): ', labeled_poi_data.head())

labeled_poi_data.to_csv('../dataset/labeledTestSet/poi_data.csv', index=False)



labeled_poi_data.head():                          region_hash    poi_ids
0  74c1c25f4b283fa74a5514307b0d0278  192205092
1  08f5b445ec6b29deba62e6fd8b0325a6   11664654
2  4b7f6f4e2bf237b6cc58f57142bea5c0   26846101
3  a814069db8d32f0fa6e188f41059c6e1   21808997
4  8316146a6f78cc6d9f113f0390859417    6828078


In [22]:
# read from the labeled data
cluster_map = pd.read_csv('../dataset/labeledTestSet/cluster_map.csv')
orders_data = pd.read_csv('../dataset/labeledTestSet/orders_data.csv')
weather_data = pd.read_csv('../dataset/labeledTestSet/weather_data.csv')
poi_data = pd.read_csv('../dataset/labeledTestSet/poi_data.csv')


In [23]:
# map time to time slot
# devide day in 10 min time slots (144 time slots)


# convert time to datetime
orders_data['time'] = pd.to_datetime(orders_data['time'])
weather_data['time'] = pd.to_datetime(weather_data['time'])

# map time to time slot
orders_data['time_slot'] = orders_data['time'].dt.hour * 6 + orders_data['time'].dt.minute // 10
weather_data['time_slot'] = weather_data['time'].dt.hour * 6 + weather_data['time'].dt.minute // 10

# map time to time slot with weekday
orders_data['weekday'] = orders_data['time'].dt.weekday
weather_data['weekday'] = weather_data['time'].dt.weekday

# remove the time column
orders_data = orders_data.drop(['time'], axis=1)
weather_data = weather_data.drop(['time'], axis=1)

print(orders_data.head())
print(weather_data.head())


                           order_id                         driver_id  \
0  a903b5f7f65f1dc7f4ee94fec74673be  dce1e90fc91ed39a7b04a22d02910a7d   
1  b7c838beaf12a2132776a1f00e016038  1f39eba5ce330f20c95177826a122e12   
2  0e0d3c93298ed59281352e34c6f1ec5a  8e4e2bc0342b3c55edea2723f6613e36   
3  1f6d0d7f68f216c4333969d6152a0a8b  5a33393e516673c8e9a065915667a30f   
4  bcf6050d9f5b270f8beb3ff80b01b435  f0232b8e45abc5ca92ca0f90fa811e7c   

                  start_region_hash                  dest_region_hash  \
0  d4ec2125aff74eded207d2d915ef682f  d4ec2125aff74eded207d2d915ef682f   
1  2407d482f0ffa22a947068f2551fe62c  2407d482f0ffa22a947068f2551fe62c   
2  b26a240205c852804ff8758628c0a86a  3a43dcdff3c0b66b1acb1644ff055f9d   
3  4725c39a5e5f4c188d382da3910b3f3f  4725c39a5e5f4c188d382da3910b3f3f   
4  dd8d3b9665536d6e05b29c2648c0e69a  a5609739c6b5c2719a3752327c5e33a7   

   time_slot  weekday  
0         44        5  
1         78        5  
2         92        5  
3        102        5  
4 

In [24]:
# group the orders data by time slot 
# aggregate count the number of orders where driver_id = NULL
# this is supply demand deficit

orders_data_grouped = orders_data.groupby(['start_region_hash', 'dest_region_hash','time_slot', 'weekday' ]).agg({'order_id': 'count'}).rename(columns={'order_id': 'supply'}).reset_index()

print(orders_data_grouped)


                      start_region_hash                  dest_region_hash  \
0      08232402614a9b48895cc3d0aeb0e9f2  08232402614a9b48895cc3d0aeb0e9f2   
1      08232402614a9b48895cc3d0aeb0e9f2  08232402614a9b48895cc3d0aeb0e9f2   
2      08232402614a9b48895cc3d0aeb0e9f2  08232402614a9b48895cc3d0aeb0e9f2   
3      08232402614a9b48895cc3d0aeb0e9f2  08232402614a9b48895cc3d0aeb0e9f2   
4      08232402614a9b48895cc3d0aeb0e9f2  08232402614a9b48895cc3d0aeb0e9f2   
...                                 ...                               ...   
74066  fff4e8465d1e12621bc361276b6217cf  fff4e8465d1e12621bc361276b6217cf   
74067  fff4e8465d1e12621bc361276b6217cf  fff4e8465d1e12621bc361276b6217cf   
74068  fff4e8465d1e12621bc361276b6217cf  fff4e8465d1e12621bc361276b6217cf   
74069  fff4e8465d1e12621bc361276b6217cf  fff4e8465d1e12621bc361276b6217cf   
74070  fff4e8465d1e12621bc361276b6217cf  fff4e8465d1e12621bc361276b6217cf   

       time_slot  weekday  supply  
0             55        6       1  
1  

In [25]:
# group the weather data by time slot
# aggregate the mean of temperature and pm25
weather_data_grouped = weather_data.groupby(['time_slot']).agg({'temperature': 'mean', 'pm25': 'mean'}).reset_index()

print(weather_data_grouped)

    time_slot  temperature       pm25
0          42     2.000000  81.333333
1          43     2.500000  75.000000
2          44     2.500000  75.000000
3          54     6.000000  30.000000
4          55     0.000000  65.000000
5          56     3.000000  73.000000
6          66     3.500000  74.500000
7          67     3.000000  71.250000
8          68     1.500000  61.000000
9          78     3.500000  63.250000
10         79     2.000000  66.500000
11         80     3.500000  63.250000
12         90     3.250000  50.250000
13         91     3.000000  52.333333
14         92     4.000000  65.500000
15        102     4.000000  58.333333
16        103     1.666667  78.333333
17        104     3.666667  58.333333
18        114     2.500000  74.000000
19        115     2.000000  66.000000
20        116     2.000000  62.000000
21        126     1.333333  84.333333
22        127     0.800000  70.000000
23        128     2.000000  74.750000
24        138     0.800000  73.800000
25        13

In [26]:
# merge the orders data and weather data
orders_weather_data = pd.merge(orders_data_grouped, weather_data_grouped, on='time_slot', how='inner')

print(orders_weather_data)


                      start_region_hash                  dest_region_hash  \
0      08232402614a9b48895cc3d0aeb0e9f2  08232402614a9b48895cc3d0aeb0e9f2   
1      08232402614a9b48895cc3d0aeb0e9f2  3e12208dd0be281c92a6ab57d9a6fb32   
2      08232402614a9b48895cc3d0aeb0e9f2  52d7b69796362a8ed1691a6cc02ddde4   
3      08232402614a9b48895cc3d0aeb0e9f2  905ac1f4c0f46a8d31ac4dc68cef54ca   
4      08232402614a9b48895cc3d0aeb0e9f2  cb6041cc08444746caf6039d8b9e43cb   
...                                 ...                               ...   
74066  fff4e8465d1e12621bc361276b6217cf  87285a66236346350541b8815c5fae94   
74067  fff4e8465d1e12621bc361276b6217cf  87285a66236346350541b8815c5fae94   
74068  fff4e8465d1e12621bc361276b6217cf  fc34648599753c9e74ab238e9a4a07ad   
74069  fff4e8465d1e12621bc361276b6217cf  fc34648599753c9e74ab238e9a4a07ad   
74070  fff4e8465d1e12621bc361276b6217cf  fff4e8465d1e12621bc361276b6217cf   

       time_slot  weekday  supply  temperature       pm25  
0             5

In [27]:
# merge the poi_list class characteristics with the cluster_map
# cluster_map: region_hash, region_id
# poi_data: district_hash, poi_ids
# merge on district_hash
cluster_map_poi = pd.merge(cluster_map, poi_data, left_on='region_hash', right_on='region_hash', how='inner')

# remove the region_hash column
# cluster_map_poi = cluster_map_poi.drop(['region_id'], axis=1)

print(cluster_map_poi.head())


                        region_hash  region_id
0  90c5a34f06ac86aee0fd70e2adce7d8a          1
1  f2c8c4bb99e6377d21de71275afd6cd2          2
2  58c7a4888306d8ff3a641d1c0feccbe3          3
3  b26a240205c852804ff8758628c0a86a          4
4  4b9e4cf2fbdc8281b8a1f9f12b80ce4d          5
                        region_hash    poi_ids
0  74c1c25f4b283fa74a5514307b0d0278  192205092
1  08f5b445ec6b29deba62e6fd8b0325a6   11664654
2  4b7f6f4e2bf237b6cc58f57142bea5c0   26846101
3  a814069db8d32f0fa6e188f41059c6e1   21808997
4  8316146a6f78cc6d9f113f0390859417    6828078
                        region_hash  region_id    poi_ids
0  90c5a34f06ac86aee0fd70e2adce7d8a          1  118257404
1  f2c8c4bb99e6377d21de71275afd6cd2          2   68155035
2  58c7a4888306d8ff3a641d1c0feccbe3          3    5013449
3  b26a240205c852804ff8758628c0a86a          4   42874231
4  4b9e4cf2fbdc8281b8a1f9f12b80ce4d          5    4327122


In [28]:
# merge the orders_data with the cluster_map_poi
# orders_weather_data: start_district_hash, time_slot, weekday, order_gap, temperature, pm25
# cluster_map_poi: region_id, poi_ids
# merge on start_district_hash
# print(orders_weather_data.head())
# print(cluster_map_poi.head())

orders_weather_cluster_map_poi = pd.merge(orders_weather_data, cluster_map_poi, left_on='start_region_hash', right_on='region_hash', how='inner')
orders_weather_cluster_map_poi=orders_weather_cluster_map_poi.rename(columns={'poi_ids': 'start_poi_ids'}).rename(columns={'region_id': 'start_region_id'})

orders_weather_cluster_map_poi = pd.merge(orders_weather_cluster_map_poi, cluster_map_poi, left_on='dest_region_hash', right_on='region_hash', how='inner')
orders_weather_cluster_map_poi=orders_weather_cluster_map_poi.rename(columns={'poi_ids': 'dest_poi_ids'}).rename(columns={'region_id': 'dest_region_id'})

# remove the start_district_hash column
orders_weather_cluster_map_poi = orders_weather_cluster_map_poi.drop(['start_region_hash', 'dest_region_hash', 'region_hash_x', 'region_hash_y'], axis=1)

print(orders_weather_cluster_map_poi)


# save the data
orders_weather_cluster_map_poi.to_csv('../dataset/processedData/orders_weather_cluster_map_poi_test.csv', index=False)


       time_slot  weekday  supply  temperature       pm25  start_region_id  \
0             55        6       1          0.0  65.000000               50   
1             56        2       1          3.0  73.000000               50   
2             56        6       4          3.0  73.000000               50   
3             66        4       2          3.5  74.500000               50   
4             66        5       1          3.5  74.500000               50   
...          ...      ...     ...          ...        ...              ...   
30918         92        5       1          4.0  65.500000               53   
30919        128        2       1          2.0  74.750000               53   
30920         91        5       1          3.0  52.333333               53   
30921         68        2       1          1.5  61.000000               53   
30922        138        0       1          0.8  73.800000               53   

       start_poi_ids  dest_region_id  dest_poi_ids  
0         